# Start Data preprocessing

In [1]:
import numpy as np
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96

In [2]:
PATH='data/nietzsche/'
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt', encoding="utf-8").read()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
vocab_list = sorted(list(set(text)))
vocab_size = len(vocab_list)+1
print("Number of unique", vocab_size)

Number of unique 85


In [4]:
vocab_list.insert(0, '\0')

In [5]:
char_to_idx = {char:i for i, char in enumerate(vocab_list)}
idx_to_char = {i: char for i, char in enumerate(vocab_list)}

In [6]:
indexed_text = [char_to_idx[char] for char in text]

In [7]:
step = 3
every_char1 = [indexed_text[i] for i in range(0, len(indexed_text) - step, step)]
every_char2 = [indexed_text[i + 1] for i in range(0, len(indexed_text) - step, step)]
every_char3 = [indexed_text[i + 2] for i in range(0, len(indexed_text) - step, step)]
output = [indexed_text[i + 3] for i in range(0, len(indexed_text) - step, step)]

In [8]:
in1 = np.stack(every_char1)
in2 = np.stack(every_char2)
in3 = np.stack(every_char3)
output = np.stack(output)

# Create 3 char model
1. Create inputs (ie. create a list of every 4th character starting at 0th, 1st, 2nd)
1. Create & train model (ie. Char3Model class - nn.Embedding, nn.Linear)
1. self.e -> self.linear_input -> F.relu x3
1. h+input -> self.linear_hidden -> F.tanh
1. ColumnarModelData.from_arrays
1. optim.Adam
1. fit()
1. get_next() - test model by getting next char

# Train Model
1. Create & train model (ie. Char3Model class - nn.Embedding, nn.Linear)
1. self.e -> self.linear_input -> F.relu x3
1. h+input -> self.linear_hidden -> F.tanh

In [9]:
n_hidden = 256
n_factors = 42

class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_factors):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factors)
        self.linear_input = nn.Linear(n_factors, n_hidden)
        self.linear_hidden = nn.Linear(n_hidden, n_hidden)
        self.linear_output = nn.Linear(n_hidden, vocab_size)
    def forward(self, x1, x2, x3):
        char1embed = self.e(x1)
        char2embed = self.e(x2)
        char3embed = self.e(x3)
        
        in1 = F.relu(self.linear_input(char1embed))
        in2 = F.relu(self.linear_input(char2embed))
        in3 = F.relu(self.linear_input(char3embed))
        
        hidden_state = V(torch.zeros(in1.size()).cuda())
        hidden_state = F.tanh(self.linear_hidden(hidden_state + in1))
        hidden_state = F.tanh(self.linear_hidden(hidden_state + in2))
        hidden_state = F.tanh(self.linear_hidden(hidden_state + in2))
        
        return F.log_softmax(self.linear_output(hidden_state))

In [10]:
model_data = ColumnarModelData.from_arrays('.', [-1], np.stack([in1, in2, in3], axis=1), output, bs=512)

In [11]:
model = Char3Model(len(vocab_list), n_factors).cuda()

In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [13]:
fit(model, model_data, 1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      2.833131   2.505615  



[array([2.50561])]

In [14]:
set_lrs(optimizer, 1e-3)

In [15]:
fit(model, model_data, 1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      2.647739   1.466757  



[array([1.46676])]

In [16]:
def get_next(inputString):
    indexes = T(np.array([char_to_idx[c] for c in inputString]))
    output = model(*VV(indexes))
    i = np.argmax(to_np(output))
    return idx_to_char[i]

In [17]:
print(get_next(' th'))

e


# Create RNN
1. Loop through and create a list of every 8th character - input char data from series 0-7 
1. Loop through and create a list of the next character in this series (ie. 9th character)
1. Create CharLoopModel -  nn.Embedding, nn.Linear, initialise torch.zeros(bs, n_hidden).cuda(), for c in cs
1. self.e -> self.linear_in -> F.relu, h+input -> linear_hidden -> tanh
1. Create CharLoopConcatModel - torch.cat((h, self.e(c)), 1)
1. Test model with get_next()

In [18]:
step = 8
train_chars = [[indexed_text[i+j] for j in range(step)] for i in range(len(indexed_text) - step)]
test_chars = [indexed_text[j+step] for j in range(len(indexed_text) - step) ]

In [19]:
train_chars = np.stack(train_chars)
test_chars = np.stack(test_chars)

In [20]:
val_idx = get_cv_idxs(len(train_chars) - step - 1)

In [22]:
model_data = ColumnarModelData.from_arrays('.', val_idx, train_chars, test_chars, bs=512)

In [23]:
class CharLoopModel(nn.Module):
    def __init__(self, vocab_size, n_factors):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factors)
        self.linear_input = nn.Linear(n_factors, n_hidden)
        self.linear_hidden = nn.Linear(n_hidden, n_hidden)
        self.linear_output = nn.Linear(n_hidden, vocab_size)
    def forward(self, *chars):
        batch_size = chars[0].size(0)
        hidden_state = V(torch.zeros(batch_size, n_hidden).cuda())
        for char in chars:
            input_activation = F.relu(self.linear_input(self.e(char)))
            hidden_state = F.tanh(self.linear_hidden(hidden_state + input_activation))
        return F.log_softmax(self.linear_output(hidden_state), dim=-1)
            

In [24]:
model = CharLoopModel(vocab_size, n_factors).cuda()

In [25]:
optimizer = optim.Adam(model.parameters(), 1e-2)

In [26]:
fit(model,model_data,1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.999152   1.989036  



[array([1.98904])]

In [27]:
set_lrs(optimizer, 1e-3)
fit(model,model_data,1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.701737   1.69438   



[array([1.69438])]

In [28]:
get_next(' th')

'e'

In [29]:
class CharConcatModel(nn.Module):
    def __init__(self, vocab_size, n_factors):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factors)
        self.linear_input = nn.Linear(n_factors + n_hidden, n_hidden)
        self.linear_hidden = nn.Linear(n_hidden, n_hidden)
        self.linear_output = nn.Linear(n_hidden, vocab_size)
    def forward(self, *chars):
        batch_size = chars[0].size(0)
        hidden_state = V(torch.zeros(batch_size, n_hidden).cuda())
        for char in chars:
            inp = torch.cat((hidden_state, self.e(char)), 1)
            inp = F.relu(self.linear_input(inp))
            hidden_state = F.tanh(self.linear_hidden(inp))
        return F.log_softmax(self.linear_output(hidden_state), dim=-1)

In [30]:
model = CharConcatModel(vocab_size, n_factors).cuda()

In [31]:
optimizer = optim.Adam(model.parameters(), 1e-3)

In [32]:
fit(model,model_data, 1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.801922   1.782424  



[array([1.78242])]

# RNN with Pytorch
1. Create CharRnn (self.rnn = nn.RNN()) - embedding, RNN, linear
1. torch.zeros() to make initialised hidden layer
1. self.e(torch.stack(cs))
1. self.rnn(input, hidden)
1. self.linear_output -> F.log_softmax
1. Test model with get_next - pass indexed text

In [34]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, n_factors):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factors)
        self.rnn = nn.RNN(n_factors, n_hidden)
        self.linear_output = nn.Linear(n_hidden, vocab_size)
    def forward(self, *chars):
        batch_size = chars[0].size(0)
        hidden_state = V(torch.zeros(1, batch_size, n_hidden ))
        inp = self.e(torch.stack(chars))
        output, hidden_state = self.rnn(inp, hidden_state)
        return F.log_softmax(self.linear_output(output[-1]), dim=-1)    

In [35]:
model = CharRNN(vocab_size, n_factors).cuda()

In [36]:
optimizer = optim.Adam(model.parameters(), 1e-3)

In [37]:
fit(model, model_data, 1, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.869353   1.840322  



[array([1.84032])]

In [38]:
set_lrs(optimizer, 1e-4)
fit(model, model_data, 3, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.774457   1.779656  
    1      1.752325   1.753175                              
    2      1.727903   1.730656                              



[array([1.73066])]

In [39]:
def get_next(chars):
    indexed_char = T(np.array([char_to_idx[char] for char in chars]))
    outp = model(*VV(indexed_char))
    i = np.argmax(to_np(outp))
    return idx_to_char[i]

In [40]:
get_next("for thos")

'e'

In [41]:
def get_next_n(chars, n):
    res = chars
    for i in range(n):
        predicted = get_next(chars)
        res = res + predicted
        chars = chars[1:] + predicted
    return res

In [42]:
get_next_n("for thos", 33)

'for those is the strenger the self-consci'

# Multi-output model

1. Get non-overlapping sets of chars
1. [[idx[i + j] for i in range(cs)]for j in range (0, up to -1 idx, step size]
1. ColumnerModelData.from_arrays()
1. Create CharSeqRnn - self.e(torch.stack(cs)) -> self.rnn() -> self.linear_output -> F.log_softmax
1. Define negative log loss (nll) - F.nll_loss()

In [43]:
step = 8
train_chars = [[indexed_text[i+j] for i in range(step)] for j in range(0, len(indexed_text) - step - 1, step)]
test_chars = [[indexed_text[i+j] for i in range(step)] for j in range(1, len(indexed_text) - step, step)]

In [44]:
train_chars = np.stack(train_chars)
test_chars = np.stack(test_chars)

In [45]:
print(train_chars[:step, :step])

[[40 42 29 30 25 27 29  1]
 [ 1  1 43 45 40 40 39 43]
 [33 38 31  2 73 61 54 73]
 [ 2 44 71 74 73 61  2 62]
 [72  2 54  2 76 68 66 54]
 [67  9  9 76 61 54 73  2]
 [73 61 58 67 24  2 33 72]
 [ 2 73 61 58 71 58  2 67]]


In [46]:
print(test_chars[:step, :step])

[[42 29 30 25 27 29  1  1]
 [ 1 43 45 40 40 39 43 33]
 [38 31  2 73 61 54 73  2]
 [44 71 74 73 61  2 62 72]
 [ 2 54  2 76 68 66 54 67]
 [ 9  9 76 61 54 73  2 73]
 [61 58 67 24  2 33 72  2]
 [73 61 58 71 58  2 67 68]]


In [47]:
val_idxs = get_cv_idxs(len(train_chars) - step - 1)

In [48]:
model_data = ColumnarModelData.from_arrays('.', val_idxs, train_chars, test_chars, bs=512)

In [49]:
class CharSeqRNN(nn.Module):
    def __init__(self, vocab_size, n_factors):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factors)
        self.rnn = nn.RNN(n_factors, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
    def forward(self, *chars):
        bs = chars[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(chars))
        outp, h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [50]:
model = CharSeqRNN(vocab_size, n_factors).cuda()

In [51]:
optimizer = optim.Adam(model.parameters(), 1e-3)

In [52]:
it = iter(model_data.trn_dl)
*xs, yt = next(it)

In [53]:
def nll_loss_seq(inp, targ):
    seq_len, bs, n_hidden = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1, n_hidden), targ)

In [54]:
fit(model, model_data, 4, optimizer, nll_loss_seq)

epoch      trn_loss   val_loss                              
    0      2.598429   2.409292  
    1      2.29071    2.200228                              
    2      2.141052   2.089137                              
    3      2.047998   2.014845                              



[array([2.01484])]

# Create RNN Types

1. Create CharSeqStatefulRNN
1. Create CharSeqStatefulRNN2
1. Create CharSeqStatefulGRU (GRU) with nn.GRU
1. Create CharrSeqStatefulLSTM (LSTM) with nn.LSTM
1. Implement Cosine Annealing ([CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)])
1. Implement get_next() & get_next_n()

In [1]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96

In [2]:
%ls {PATH}

models/  nietzsche.txt  trn/  val/


In [5]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
model_data = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(model_data.trn_dl), model_data.nt, len(model_data.trn_ds), len(model_data.trn_ds[0].text)

(947, 55, 1, 485749)

In [6]:
def repackage_var(h):
    return Variable(h.data) if type(h) == Variable else tuple(repackage_var(v) for v in h)

In [8]:
class CharSeqStatefulRNN (nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.hidden_init(bs)
        
    def forward(self, chars):
        bs = chars[0].size(0)
        if self.hidden.size(1) != bs: self.hidden_init(bs)
        outp, h = self.rnn(self.e(chars), self.hidden)
        self.hidden = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)

    def hidden_init(self, bs):
        self.hidden = V(torch.zeros(1, bs, n_hidden))

In [9]:
model = CharSeqStatefulRNN(model_data.nt, n_fac, 512).cuda()

In [11]:
optimizer = optim.Adam(model.parameters(), 1e-3)

In [12]:
fit(model, model_data, 3, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.875788   1.859158  
    1      1.689418   1.713159                               
    2      1.604204   1.643834                               



[array([1.64383])]

In [13]:
set_lrs(optimizer, 1e-4)
fit(model, model_data, 3, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.529106   1.598335  
    1      1.52269    1.589979                               
    2      1.511332   1.58347                                



[array([1.58347])]

In [26]:
def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [44]:
class CharSeqStatefulRNN2 (nn.Module):
    def __init__ (self, vocab_size, n_factors, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_factors)
        self.rnn = nn.RNNCell(n_factors, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.hidden_init(bs)

    def forward(self, chars):
        bs = chars[0].size(0)
        if self.hidden.size(1) != bs: self.hidden_init(bs)
        outp = []
        o = self.hidden
        for c in chars:
            o = self.rnn(self.e(c), o)
            outp.append(o)
        self.hidden = repackage_var(o)
        return F.log_softmax(self.l_out(torch.stack(outp)), dim=-1).view(-1, self.vocab_size)
    
    def hidden_init(self, bs):
        self.hidden = V(torch.zeros(1, bs, n_hidden))

In [45]:
model = CharSeqStatefulRNN2(model_data.nt, n_fac, 512).cuda() #md.nt is len(field.vocab)

In [46]:
optimizer = optim.Adam(model.parameters(), 1e-3)

In [49]:
fit(model, model_data, 9, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.381398   1.518046  
    1      1.37411    1.511216                               
    2      1.36493    1.509865                               
    3      1.357007   1.508882                               
    4      1.361466   1.508638                               
    5      1.35379    1.510122                               
    6      1.345629   1.508762                               
    7      1.348687   1.510139                               
    8      1.340952   1.511155                               



[array([1.51115])]

In [75]:
class CharSeqStatefulGRU (nn.Module):
    def __init__(self, vocab_size, n_factors, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_factors)
        self.gru = nn.GRU(n_factors, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.hidden_init(bs)
        
    def forward(self, chars):
        bs = chars[0].size(0)
        if self.hidden.size(1) != bs: self.hidden_init(bs)
        outp, h = self.gru(self.e(chars), self.hidden)    
        self.hidden = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def hidden_init(self, bs):
        self.hidden = V(torch.zeros(1, bs, n_hidden))

In [77]:
model = CharSeqStatefulGRU(model_data.nt, n_fac, 512).cuda()
optimizer = optim.Adam(model.parameters(), 1e-3)

In [78]:
fit(model, model_data, 3, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.740669   1.731596  
    1      1.554523   1.585686                               
    2      1.467744   1.518437                               



[array([1.51844])]

In [82]:
fit(model, model_data, 3, optimizer, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.329577   1.45386   
    1      1.306259   1.451589                               
    2      1.294584   1.449693                               



[array([1.44969])]

In [97]:
class CharSeqStatefulLSTM (nn.Module):
    def __init__(self, vocab_size, n_factors, bs, nl):
        super().__init__()
        self.vocab_size = vocab_size
        self.nl = nl
        self.e = nn.Embedding(vocab_size, n_factors)
        self.lstm = nn.LSTM(n_factors, n_hidden, nl, dropout=0.5) #nl = number of layers
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.hidden_init(bs)

    def forward(self, chars):
        bs = chars[0].size(0)
        if self.hidden[0].size(1) != bs: self.hidden_init(bs)
        outp, h = self.lstm(self.e(chars), self.hidden)
        self.hidden = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
        
    def hidden_init(self, bs):
        self.hidden = (V(torch.zeros(self.nl, bs, n_hidden)), 
                       V(torch.zeros(self.nl, bs, n_hidden)))
        #hidden layer initialised as a tuple of 2 0 matrices

In [98]:
model = CharSeqStatefulLSTM(model_data.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, model, 1e-2, 1e-5)

In [99]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [100]:
fit(model, model_data, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss                               
    0      1.77631    1.687049  
    1      1.673802   1.595266                               



[array([1.59527])]

In [101]:
?CosAnneal

In [105]:
#Call backs to do Cosine Annealing
on_end = lambda sched, cycle: save_model(model, f'{PATH}models/cycle_{cycle}') #Save model
call_back = [CosAnneal(lo, len(model_data.trn_dl), cycle_mult=2, on_cycle_end=on_end)] #trn_dl is length of the dataloader
fit(model, model_data, 2**6-1, lo.opt, F.nll_loss, callbacks = call_back)

epoch      trn_loss   val_loss                               
    0      1.187258   1.31315   
    1      1.186601   1.313008                               
    2      1.186906   1.312391                               
    3      1.187094   1.313229                               
    4      1.190092   1.312537                               
    5      1.18726    1.311848                               
    6      1.1829     1.311563                               
    7      1.187133   1.312235                               
    8      1.186393   1.312395                               
    9      1.185543   1.312864                               
    10     1.186253   1.312902                               
    11     1.181841   1.312306                               
    12     1.182119   1.312219                               
    13     1.177267   1.312097                               
    14     1.175407   1.311875                               
    15     1.179057   1.313027       

[array([1.31566])]

In [106]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = model(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [107]:
get_next('for thos')

'e'

In [108]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [110]:
print(get_next_n('for thos', 400))

for those who, as an investignors intoo future. learned beaut of their genesis a german quite to be a timeof the habit so long of the lurrowers, amorn of feelings ins"faces." first and down the dreams: one has been here.47. perhaps pleasure in3rage that they arethe thinking of all that comes, indeed it con taste. in one--"what they have, holy advincious more their so far to itself is almost by itself--how
